<a href="https://colab.research.google.com/github/zidanesouza/SBSPS-Challenge-4551-Sentiment-Analysis-of-COVID-19-Tweets-Visualization-Dashboard/blob/master/get_CSV_for_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
# import lots of stuff
import sys
import os
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

import pandas as pd

from datetime import datetime, timedelta
from IPython.display import clear_output

from os import path


In [70]:
#@title Enter Twitter Credentials
TWITTER_KEY = '' #@param {type:"string"}
TWITTER_SECRET_KEY = '' #@param {type:"string"}
access_key=''#@param {type:"string"}
access_secret=''#@param {type:"string"}

In [71]:
# Authenticate
auth = tweepy.OAuthHandler(TWITTER_KEY, TWITTER_SECRET_KEY)
auth.set_access_token(access_key,access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)


In [72]:
#@title Twitter Search API Inputs
#@markdown ### Enter Search Query:
searchQuery = 'IndiaFightsCorona' #@param {type:"string"}
#@markdown ### Enter Max Tweets To Scrape:
#@markdown #### The Twitter API Rate Limit (currently) is 45,000 tweets every 15 minutes.
maxTweets = 200 #@param {type:"slider", min:0, max:45000, step:100}
Filter_Retweets = True #@param {type:"boolean"}

tweetsPerQry = 10  # this is the max the API permits
tweet_lst = []

if Filter_Retweets:
  searchQuery = searchQuery + ' -filter:retweets'  # to exclude retweets

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -10000000000

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang="en")
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", max_id=str(max_id - 1),
                                        since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
            break
        for tweet in new_tweets:
          if hasattr(tweet, 'reply_count'):
            reply_count = tweet.reply_count
          else:
            reply_count = 0
          if hasattr(tweet, 'retweeted'):
            retweeted = tweet.retweeted
          else:
            retweeted = "NA"
            
          # fixup search query to get topic
          topic = searchQuery[:searchQuery.find('-')].capitalize().strip()
          
          # fixup date
          tweetDate = tweet.created_at.date()
          
          tweet_lst.append([tweetDate, topic, 
                      tweet.id, tweet.user.screen_name, tweet.user.name, tweet.text, tweet.favorite_count, 
                      reply_count, tweet.retweet_count, retweeted])

        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break

clear_output()
print("Downloaded {0} tweets".format(tweetCount))

Downloaded 0 tweets


In [75]:
pd.set_option('display.max_colwidth', -1)

# load it into a pandas dataframe
tweet_df = pd.DataFrame(tweet_lst, columns=['tweet_dt', 'topic', 'id', 'username', 'name', 'tweet', 'like_count', 'reply_count', 'retweet_count', 'retweeted'])
tweet_df.to_csv('tweet_text_IndiaFightsCorona.csv')
print(tweet_df['tweet'].to_csv('tweet_text_IndiaFightsCorona.csv'))# common file for both txt snd semnt analysis

None


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [73]:
# Hello Zidane, manually change the file to required file name
# example change "tweet_text_lockdown.csv"  to "tweet_text_IndiaFightsCorona.csv" to get its tweet and sentiment etc

import csv
with open('tweet_text_IndiaFightsCorona.csv') as csvfile:
      read_tweet = csv.reader(csvfile)
      for row in read_tweet:
           print(row[1])
           print("ANSWER")
           print( TextBlob(row[1]).sentiment)
           print(" ")
           

tweet
ANSWER
Sentiment(polarity=0.0, subjectivity=0.0)
 


In [64]:
 
TextBlob("@schrodie_twts @ankitatweetsyo *Gets hug*\n\n*Tests positive for covid-19* 🌝🌝").sentiment
       

Sentiment(polarity=0.22727272727272727, subjectivity=0.5454545454545454)